# ДЗ №6 - автокодировщики для идентификации аномалий

В этом ДЗ вам предстоит применить модель сврточного автокодировщика для идентификации аномалий в данных.

Для этого вам потребуется создать сверточный автокодировщик, обучить его и применить к тестовым данным.

Основная идея фильтрации аномалий состоит в том, что экземпляры выборки, являющиеся аномалиями, сильно отличаются от всех остальных объектов. Кроме того, их мало по сранению с размером всей выборки.
Этот набор факторов приводит к тому, что автокодировщик, обученный на данных тренировочной выборки, будет довольно плохо восстанавливать примеры-аномалии. То есть, значения функции потерь на таких примерах ожидается нетипично высоким.

In [ ]:
# Эту ячейку следует выоплнять в окружении, в котором еще не установлены необходимые библиотеки. В подготовленном окружении эту ячейку можно пропустить.
!pip3 install torch torchvision numpy matplotlib

In [3]:
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from PIL import Image

from typing import Tuple, List, Type, Dict, Any

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Свёрточный автокодировщик (convolutional autoencoder, CAE)

Данными в этой задаче будут все так же набор рукописных цифр MNIST. Однако некоторые экземпляры тестовой выборки оказываются испорченными. Ваша цель - найти эти экземпляры в предположении, что они представляют собой аномалии.

Данные MNIST с дефектами нужно скачать в виде файла по <a href="https://www.dropbox.com/s/r7mgjn83y9ygpzq/mnist_corrupted.npz">ссылке</a>

Прежде всего следует построить и обучить свёрточный автокодировщик.

>Кодирующая часть автокодировщика (encoder, кодировщик) может состоять из сверточных слоев (convolutional layers) и слоев субдискретизации (pooling layers), но может быть и сложнее. Здесь предлагается применить ваши знания относительно возможной структуры сверточных сетей. Кодировщик, будучи обученным, позволяет извлечь скрытое представление (hidden representation, embeddings) входных примеров, содержащее достаточно информации для восстановления этих примеров декодером.

> Декодер (decoder) может состоять из слоев типа **transpose convolution** и операций масштабирования (upsampling), но также, как и кодировщик, может быть сложнее. Декодер должен восстанавливать примеры, руководствуюясь их векторами скрытого представления.

<img src='imgs/autoencoder_1.png' />

### Скрытое представление (hidden representation, compressed representation)

Скрытое представление может содержать семантически насыщенную информацию о входных примерах. С использованием этих данных можно проводить фильтрацию шума в примерах, восстанавливать сами примеры, и иногда даже проводить некоторые операции в семантическом пространстве.

В предположении, что файл данных `mnist_corrupted.npz` загружен и находится в той же директории, что и этот нотбук, генераторы данных можно описать следующим образом:

In [5]:
class DS(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]

        if self.transform:
            x = Image.fromarray(x.astype(np.uint8))
            x = self.transform(x)

        return x

    def __len__(self):
        return len(self.data)

In [ ]:
mnist = np.load('./mnist_corrupted.npz')
mnist_train_samples = mnist['x_train']
mnist_test_samples = mnist['x_test']
train_dataset = DS(mnist_train_samples, train_transforms)
val_dataset = DS(mnist_test_samples, val_transforms)

### Визуализация исходных данных

Как и в любой задаче, имеет смысл визуализировать исходные данные, чтобы понимать, с чем мы имеем дело

In [ ]:
indices = np.random.randint(0, len(train_dataset), size=8)

fig, axes = plt.subplots(nrows=1, ncols=8, figsize=(8, 2), dpi=300)
for i, ax in enumerate(axes):
    sample_index = indices[i]
    sample = train_dataset[sample_index]
    ax.imshow(np.squeeze(sample.cpu().numpy()), cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()
fig.patch.set_facecolor('white')

---
## Свёрточный автокодировщик

#### Кодировщик (Encoder)
Кодировщик можно реализовать в подходе AlexNet или VGG: сверточные (convolutional) слои чередуются со слоями субдискретизации (pooling). Последние применяются для снижения пространственных размерностей промежуточных представлений входных примеров. Нередко после сверточной части добавляют дополнительные полносвязные слои, позволяющие еще сильнее снизить размерность скрытого представления, извлекаемого кодировщиком.

Предлагаемая структура кодировщика не единственно верная. Можно реализовывать и другие.

#### Декодер

Декодер должен преобразовать вектор скрытого представления (тензор ранга 1) в изображение, реконструкцию входного примера. Для этого следует вектор скрытого представления перевести в ранг 2 (например, операцией `.view()`). После этого следует последовательно применять операции Transpose Convolution (`torch.nn.ConvTranspose2d`) и масштабирования (upsampling, а именно `torch.nn.functional.interpolate`). В некоторых случаях применяют `torch.nn.ConvTranspose2d` с аргументом `stride=2` или больше. Однако такое использование может привести в т.н. ["эффекту шахматной доски"](https://distill.pub/2016/deconv-checkerboard/). Рекомендуемым вариантом сейчас считается применение масштабирования типа билинейного или бикубического.

Результатом работы декодера должно получиться изображение, по размеру совпадающее с входным примером, то есть, 28x28.

Не следует забывать, что одной из целей применения автокодировщиков является снижение размерности примеров с сохранением ключевой информации. Экспериментируйте с количеством слоев и размерностью скрытого представления! Попробуйте снизить его до 2 или вообще до 1. Хорошо ли будут воспроизводиться примеры выборки?

### Transpose Convolutions

В этом ДЗ в декодере предлагается использовать слои типа **transposed convolutional**. Они работают практически так же, как свёрточные слои, но "задом наперед". Например, ядро размером 3x3 в случае свёрточной операции дает в результате одно значение. Операция **transposed convolutional**, наоборот, одно значение входного представления трансформирует в патч размером с его ядро (3x3). В PyTorch есть уже готовая реализация слоев [`nn.ConvTranspose2d`](https://pytorch.org/docs/stable/nn.html#convtranspose2d).

Повторимся, альтернативой использованию **transposed convolutional layer** с аргументом `stride=2` или больше может быть применение операций изменения размера (resizing) с интерполяцией типа "nearest neighbor", "bilinear" или "bicubic" и применением свёрточной операции к результату.

### Задание 1:  Описать класс нейросети-автокодировщика, описываемой в этом задании.

In [ ]:
# define the NN architecture
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        ## слои кодировщика ##
        # YOUR CODE HERE
        
        ## слои декодера ##
        # YOUR CODE HERE


    def forward(self, x):
        ## применить операции кодировщика ##
        
        ## применить операции декодера ##
        ## На всех скрытых слоях применяйте функцию активации ReLU
        ## В случае, если данные отнормированы к диапазону [0,1], активацией последнего слоя может быть sigmoid.
        ## Подумайте, какая может быть функция активации в других случаях. Реализуйте подходящий вариант
                        
        return x

In [ ]:
model = ConvAutoencoder()
print(model)

In [ ]:
model = model.to(device)

### Задание 2: Напишите пайплайн для предобработки и аугументации данных.

В `torchvision.transforms` есть готовые реализации большинства распространённых техник, если вы хотите добавить что-то своё, вы можете воспользоваться `torchvision.transforms.Lambda` или встроить аугментации на этапе подготовки данных в классе `DS`.

In [ ]:
train_transforms = torchvision.transforms.Compose([
    #YOUR CODE HERE
    torchvision.transforms.ToTensor()])

val_transforms = torchvision.transforms.Compose([
    #YOUR CODE HERE
    torchvision.transforms.ToTensor()])

Всегда имеет смысл посмотреть, как происходит предобработка данных, и как происходит обработка данных нейросетью (если это возможно). В этом ДЗ предлагается визуализировать произвольные примеры из обучающей выборки, а также один из произвольных примеров, обработанных только что созданной (но не обученной) моделью.

### Задание 3: отобразите несколько произвольных примеров обучающей выборки.

In [6]:
## YOUR CODE HERE ##

### Задание 4: отобразите один произвольный пример обучающей выборки и результат вычисления нейросети на этом примере.

In [ ]:
example_index = int(np.random.randint(0, len(train_dataset), size=1))
example = train_dataset[example_index]

## compute model output for this example;
## Transfer the result to CPU and convrt it from tensor to numpy array
example_transformed = None
## YOUR CODE HERE

In [ ]:
## Now plot the example and the transformed example
## YOUR CODE HERE

### Обучение модели

Теперь, когда вы реализовали модель и подготовили данные, можно приступить к непосредственному обучению модели.

Костяк функции обучения написан ниже, далее вы должны будете реализовать ключевые части этого алгоритма

In [ ]:
def train_model(model: torch.nn.Module, 
                train_dataset: torch.utils.data.Dataset,
                val_dataset: torch.utils.data.Dataset,
                loss_function: torch.nn.Module = torch.nn.CrossEntropyLoss(),
                optimizer_class: Type[torch.optim.Optimizer] = torch.optim.Adam,
                optimizer_params: Dict = {},
                lr_scheduler_class: Any = torch.optim.lr_scheduler.ExponentialLR,
                lr_scheduler_params: Dict = {},
                batch_size = 64,
                max_epochs = 100,
                early_stopping_patience = 10
):
    optimizer = optimizer_class(model.parameters(), **optimizer_params)
    lr_scheduler = lr_scheduler_class(optimizer, **lr_scheduler_params)
    
    train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)

    best_val_loss = None
    best_epoch = None
    
    for epoch in range(max_epochs):
        print(f'Epoch {epoch+1} of {max_epochs}')
        train_single_epoch(model, optimizer, loss_function, train_loader)
        val_metrics = validate_single_epoch(model, loss_function, val_loader)
        print(f'Validation metrics: \n{val_metrics}')

        lr_scheduler.step(val_metrics['loss'])
        
        if best_val_loss is None or best_val_loss > val_metrics['loss']:
            print(f'Best model yet, saving')
            best_val_loss = val_metrics['loss']
            best_epoch = epoch
#             torch.save(model, './best_model.pth')
            
        if epoch - best_epoch > early_stopping_patience:
            print('Early stopping triggered')
            return

### Задание 5:  Реализуйте функцию, производящую обучение сети на протяжении одной эпохи ( полного прохода по всей обучающей выборке ). На вход будет приходить модель, оптимизатор, функция потерь и объект типа `DataLoader`.
> ВНИМАНИЕ!!! В задаче обучения автокодировщика нет меток-цифр. Есть только входные примеры. При итерировании по `data_loader` вы будете получать только сами примеры! Подумайте, что должно выступать в качестве целевой переменной, когда вы вычисляете функцию потерь.

In [ ]:
def train_single_epoch(model: torch.nn.Module,
                       optimizer: torch.optim.Optimizer, 
                       loss_function: torch.nn.Module, 
                       data_loader: torch.utils.data.DataLoader):
    
    model.train()
    ## YOUR CODE HERE

### Задание 6:  Реализуйте функцию производящую расчёт функции потерь на тестовой выборке.  На вход будет приходить модель, функция потерь и DataLoader. На выходе ожидается словарь с вида:
```
{
    'loss': <среднее значение функции потерь>,
    'accuracy': <среднее значение точности модели>
}
```

In [ ]:
def validate_single_epoch(model: torch.nn.Module,
                          loss_function: torch.nn.Module, 
                          data_loader: torch.utils.data.DataLoader):
    
    model.eval()
    test_loss = 0.0
    
    ## YOUR CODE HERE
    
    return {
        'loss': test_loss / len(data_loader.dataset)
    }

Если вы корректно реализовали все предыдущие шаги и ваша модель имеет достаточное количество обучаемых параметров, то в следующей ячейке должен пойти процесс обучения.

### Задание 7: придумайте функцию потерь.

Обратите внимание, что в предложенном скелетном коде функция потерь по умолчанию прописана неверно. Вы, скорее всего, не сможете обучить автокодировщик с этой функцией потерь. Подумайте, какая должна быть функция потерь при условии, что она должна оценивать качество воспроизведения значений в каждом отдельном пикселе изображения. Впишите в ячейке ниже правильную функцию потерь. Подумайте, можно ли использовать уже предложенную функцию потерь, и что нужно сделать с данными, чтобы с ней можно было обучить вашу модель.

In [ ]:
train_model(model, 
            train_dataset=train_dataset, 
            val_dataset=val_dataset, 
            loss_function=None, 
            initial_lr=0.0001)

## Проверка результатов

Посмотрите, как ваш обученный автокодировщик преобразует входные примеры. В ячейке ниже приведен код для отображения произвольной пары пример-реконструкция.

In [ ]:
index = int(np.random.randint(0, len(train_dataset), size=1))
sample = train_dataset[index][0]
sample_np = np.squeeze(sample.detach().cpu().numpy())
sample_ae = model(sample.view(1,1,28,28).to(device))
sample_ae_np = np.squeeze(sample_ae.detach().cpu().numpy())

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(4, 2), dpi=200)
for i, ax in enumerate(axes):
    img = sample_np if i==0 else sample_ae_np
    ax.imshow(img, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()
fig.patch.set_facecolor('white')
plt.imshow(img, cmap='gray')

## Идентификация аномалий.

Идея идентификации аномалий состоит в том, чтобы разделить "обычные" экземпляры и "необычные" по значению функции потерь автокодировщика на этих примерах. Предполагается, что автокодировщик, обученный на обычных примерах не будет способен достаточно точно воспроизвести необычные примеры. То есть, значение функции потерь на необычных экземплярах будет большим. В этом ДЗ предлагается найти все экземпляры-выбросы, встречающиеся в тестовой выборке, руководствуюясь только значениями функции потерь автокодировщика. Для этого на всех объектах тестовой выборки следует вычислить функцию потерь обученного автокодировщика, и определить, какие экземпляры являются аномальными.

В качестве решения всего задания следует получить список значений 0 или 1, соответствующих объектам тестовой выборки. Признак `1` означает, что этот объект является аномалией, `0` - означает, что объект обычный.

Например, следующий список `[1,1,1,0,0,0,0,0,0,0,1,0]` означает, что в выборке из 12 объектов тестовой выборки аномалиями считаются первые три и предпоследний. Остальные считаются обычными.

> ВНИМАНИЕ! Сопоставление при проверке будет производиться только по номерам объектов в тестовой выборке. Поэтому выборку при вычислении функции потерь не следует перемешивать. То есть, при создании загрузчика данных `torch.utils.data.DataLoader` аргумент перемешивания должен быть выключен: `shuffle=False`

### Задание 8: примените обученную модель автокодировщика к данным тестовой выборки. Вычислите функцию потерь на каждом объекте тестовой выборки.

In [ ]:
model.eval()
test_dataset = DS(mnist_test_samples, val_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1, shuffle=False)
loss_function = None

losses = []

with torch.no_grad():
    with tqdm(total=len(test_loader)) as pbar:
        for data in test_loader:
            ## здесь следует вычислить значения функции потерь для всех элементов тестовой выборки.
            curr_loss = 0.0
            ## YOUR CODE HERE
            
            losses.append(curr_loss)
            pbar.update(1)

## Анализ значений функции потерь
Проанализируйте распределение значений функции потерь и найдите объекты, на которых она слишком большая.

### Задание 9:
- Отобразите гистограмму значений функции потерь. Сделайте выводы (напишите ТЕКСТ) относительно значений для обычных объектов и аномалий.
- Найдите объекты-аномалии, отобразите их.
- Вычислите на них обученный вами автокодировщик. Отобразите рядом объекты-аномалии и их реконструкцию, вычисленную вашим автокодировщиком.

In [ ]:
## YOUR CODE HERE

### Задание 10: создайте файл маркировки аномалий

В этом задании требуется записать в файл признаки аномальности для всех объектов тестовой выборки в том порядке, в котором эти объекты идут в выборке. Это должен быть просто текстовый файл. В нем не должно быть никаких заголовков, никаких дополнительных символов. Только `0` или `1`

<br />
пример содержимого файла (для выборки длиной 244 объекта, из которых 6 оказались помечены как аномалии):

`0000000000000010000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000001100000000`

<br /><br />
Финальным решением этого ДЗ является этот файл. Его нужно сдать вместе с ноутбуком с вашим кодом.

In [ ]:
## YOUR CODE HERE